<div>
     <div>
        <img src="./report/isel_logo.png" width="400" height="400" align="left">
    </div>
    <div>
        <h2>Área Departamental de Engenharia de Eletrónica e Telecomunicações e de Computadores</h2>
        <p>Trabalho prático 3</p>
        <p>Autor:	44598	André L. A. Q. de Oliveira</p>
        <p>Unidade Curricular Compressão de Sinais Multimédia</p>
        <p>Professor: André Lourenço</p>
        <p>06 - Junho - 2021</p>
    </div>
</div>

### <a id="index"></a>

# Index
- [Compressão JPEG](#compressao_jpeg)
- [I/O Utilities](#io_utilities)
    - [InputBitReader](#input_bit_reader)
- [Conclusões](#conclusoes)

# Importar bibliotecas

In [41]:
import os
from time import time
import cv2
import numpy as np
import matplotlib.pyplot as plt

cwd = os.getcwd() # current work diretory

# Importar dados

In [42]:
lena_gray = cv2.imread(f"{cwd}/input_data/lena.tif", cv2.IMREAD_GRAYSCALE)
lena_color = cv2.imread(f"{cwd}/input_data/lena.tif")

lena_block_1 = lena_gray[0:8,0:8]
lena_block_2 = lena_gray[0:8,8:16]

<a id="compressao_jpeg"></a>

# Compressão JPEG

# Tabelas da norma ITU T.81 - JPEG standard

In [160]:
# tabela K1 - Luminance quantize Matrix  
Q = np.zeros((8, 8))
Q[0] = [ 16,  11,  10,  16,  24,  40,  51,  61]
Q[1] = [ 12,  12,  14,  19,  26,  58,  60,  55]
Q[2] = [ 14,  13,  16,  24,  40,  57,  69,  56]
Q[3] = [ 14,  17,  22,  29,  51,  87,  80,  62]
Q[4] = [ 18,  22,  37,  56,  68, 109, 103,  77]
Q[5] = [ 24,  35,  55,  64,  81, 104, 113,  92]
Q[6] = [ 49,  64,  78,  87, 103, 121, 120, 101]
Q[7] = [ 72,  92,  95,  98, 112, 100, 103,  99]

# Tabela K3 
# Table for luminance DC coefficient differences
K3 = dict()
K3[0] = "00"
K3[1] = "010"
K3[2] = "011"
K3[3] = "100"
K3[4] = "101"
K3[5] = "110"
K3[6] = "1110"
K3[7] = "11110"
K3[8] = "111110"
K3[9] = "1111110"
K3[10] = "11111110"
K3[11] = "111111110"

# Tabela K5 
# Table for luminance AC coefficients
K5 = dict()
K5[(0, 0)] = "1010"
K5[(0, 1)] = "00"
K5[(0, 2)] = "01"
K5[(0, 3)] = "100"
K5[(0, 4)] = "1011"
K5[(0, 5)] = "11010"
K5[(0, 6)] = "1111000"
K5[(0, 7)] = "11111000"
K5[(0, 8)] = "1111110110"
K5[(0, 9)] = "1111111110000010"
K5[(0,10)] = "1111111110000011"

K5[(1, 1)] = "1100"
K5[(1, 2)] = "11011"
K5[(1, 3)] = "1111001"
K5[(1, 4)] = "111110110"
K5[(1, 5)] = "11111110110"
K5[(1, 6)] = "1111111110000100"
K5[(1, 7)] = "1111111110000101"
K5[(1, 8)] = "1111111110000110"
K5[(1, 9)] = "1111111110000111"
K5[(1,10)] = "1111111110001000"

K5[(2, 1)] = "11100"
K5[(2, 2)] = "11111001"
K5[(2, 3)] = "1111110111"
K5[(2, 4)] = "111111110100"
K5[(2, 5)] = "1111111110001001"
K5[(2, 6)] = "1111111110001010"
K5[(2, 7)] = "1111111110001011"
K5[(2, 8)] = "1111111110001100"
K5[(2, 9)] = "1111111110001101"
K5[(2,10)] = "1111111110001110"

K5[(3, 1)] = "111010"
K5[(3, 2)] = "111110111"
K5[(3, 3)] = "111111110101"
K5[(3, 4)] = "1111111110001111"
K5[(3, 5)] = "1111111110010000"
K5[(3, 6)] = "1111111110010001"
K5[(3, 7)] = "1111111110010010"
K5[(3, 8)] = "1111111110010011"
K5[(3, 9)] = "1111111110010100"
K5[(3,10)] = "1111111110010101"

K5[(4, 1)] = "111011"
K5[(4, 2)] = "1111111000"
K5[(4, 3)] = "1111111110010110"
K5[(4, 4)] = "1111111110010111"
K5[(4, 5)] = "1111111110011000"
K5[(4, 6)] = "1111111110011001"
K5[(4, 7)] = "1111111110011010"
K5[(4, 8)] = "1111111110011011"
K5[(4, 9)] = "1111111110011100"
K5[(4,10)] = "1111111110011101"

K5[(5, 1)] = "1111010"
K5[(5, 2)] = "11111110111"
K5[(5, 3)] = "1111111110011110"
K5[(5, 4)] = "1111111110011111"
K5[(5, 5)] = "1111111110100000"
K5[(5, 6)] = "1111111110100001"
K5[(5, 7)] = "1111111110100010"
K5[(5, 8)] = "1111111110100011"
K5[(5, 9)] = "1111111110100100"
K5[(5,10)] = "1111111110100101"

K5[(6, 1)] = "1111011"
K5[(6, 2)] = "111111110110"
K5[(6, 3)] = "1111111110100110"
K5[(6, 4)] = "1111111110100111"
K5[(6, 5)] = "1111111110101000"
K5[(6, 6)] = "1111111110101001"
K5[(6, 7)] = "1111111110101010"
K5[(6, 8)] = "1111111110101011"
K5[(6, 9)] = "1111111110101100"
K5[(6,10)] = "1111111110101101"

K5[(7, 1)] = "11111010"
K5[(7, 2)] = "111111110111"
K5[(7, 3)] = "1111111110101110"
K5[(7, 4)] = "1111111110101111"
K5[(7, 5)] = "1111111110110000"
K5[(7, 6)] = "1111111110110001"
K5[(7, 7)] = "1111111110110010"
K5[(7, 8)] = "1111111110110011"
K5[(7, 9)] = "1111111110110100"
K5[(7,10)] = "1111111110110101"

K5[(8, 1)] = "111111000"
K5[(8, 2)] = "111111111000000"
K5[(8, 3)] = "1111111110110110"
K5[(8, 4)] = "1111111110110111"
K5[(8, 5)] = "1111111110111000"
K5[(8, 6)] = "1111111110111001"
K5[(8, 7)] = "1111111110111010"
K5[(8, 8)] = "1111111110111011"
K5[(8, 9)] = "1111111110111100"
K5[(8,10)] = "1111111110111101"

K5[(9, 1)] = "111111001"
K5[(9, 2)] = "1111111110111110"
K5[(9, 3)] = "1111111110111111"
K5[(9, 4)] = "1111111111000000"
K5[(9, 5)] = "1111111111000001"
K5[(9, 6)] = "1111111111000010"
K5[(9, 7)] = "1111111111000011"
K5[(9, 8)] = "1111111111000100"
K5[(9, 9)] = "1111111111000101"
K5[(9,10)] = "1111111111000110"

K5[(10, 1)] = "111111010"
K5[(10, 2)] = "1111111111000111"
K5[(10, 3)] = "1111111111001000"
K5[(10, 4)] = "1111111111001001"
K5[(10, 5)] = "1111111111001010"
K5[(10, 6)] = "1111111111001011"
K5[(10, 7)] = "1111111111001100"
K5[(10, 8)] = "1111111111001101"
K5[(10, 9)] = "1111111111001110"
K5[(10,10)] = "1111111111001111"

K5[(11, 1)] = "1111111001"
K5[(11, 2)] = "1111111111010000"
K5[(11, 3)] = "1111111111010001"
K5[(11, 4)] = "1111111111010010"
K5[(11, 5)] = "1111111111010011"
K5[(11, 6)] = "1111111111010100"
K5[(11, 7)] = "1111111111010101"
K5[(11, 8)] = "1111111111010110"
K5[(11, 9)] = "1111111111010111"
K5[(11,10)] = "1111111111011000"

K5[(12, 1)] = "1111111010"
K5[(12, 2)] = "1111111111011001"
K5[(12, 3)] = "1111111111011010"
K5[(12, 4)] = "1111111111011011"
K5[(12, 5)] = "1111111111011100"
K5[(12, 6)] = "1111111111011101"
K5[(12, 7)] = "1111111111011110"
K5[(12, 8)] = "1111111111011111"
K5[(12, 9)] = "1111111111100000"
K5[(12,10)] = "1111111111100001"

K5[(13, 1)] = "11111111000"
K5[(13, 2)] = "1111111111100010"
K5[(13, 3)] = "1111111111100011"
K5[(13, 4)] = "1111111111100100"
K5[(13, 5)] = "1111111111100101"
K5[(13, 6)] = "1111111111100110"
K5[(13, 7)] = "1111111111100111"
K5[(13, 8)] = "1111111111101000"
K5[(13, 9)] = "1111111111101001"
K5[(13,10)] = "1111111111101010"

K5[(14, 1)] = "1111111111101011"
K5[(14, 2)] = "1111111111101100"
K5[(14, 3)] = "1111111111101101"
K5[(14, 4)] = "1111111111101110"
K5[(14, 5)] = "1111111111101111"
K5[(14, 6)] = "1111111111110000"
K5[(14, 7)] = "1111111111110001"
K5[(14, 8)] = "1111111111110010"
K5[(14, 9)] = "1111111111110011"
K5[(14,10)] = "1111111111110100"

K5[(15, 0)] = "11111111001"
K5[(15, 1)] = "1111111111110101"
K5[(15, 2)] = "1111111111110110"
K5[(15, 3)] = "1111111111110111"
K5[(15, 4)] = "1111111111111000"
K5[(15, 5)] = "1111111111111001"
K5[(15, 6)] = "1111111111111010"
K5[(15, 7)] = "1111111111111011"
K5[(15, 8)] = "1111111111111100"
K5[(15, 9)] = "1111111111111101"
K5[(15,10)] = "1111111111111110"

# ordem zig-zag 
zigzag = np.zeros((8, 8))
zigzag[0] = [ 0,  1,  5,  6, 14, 15, 27, 28]
zigzag[1] = [ 2,  4,  7, 13, 16, 26, 29, 42]
zigzag[2] = [ 3,  8, 12, 17, 25, 30, 41, 43]
zigzag[3] = [ 9, 11, 18, 24, 31, 40, 44, 53]
zigzag[4] = [10, 19, 23, 32, 39, 45, 52, 54]
zigzag[5] = [20, 22, 33, 38, 46, 51, 55, 60]
zigzag[6] = [21, 34, 37, 47, 50, 56, 59, 61]
zigzag[7] = [35, 36, 48, 49, 57, 58, 62, 63]
ind_zz = zigzag.reshape((64),order='F').astype('int16')

<a id="indexing_using_zigzag"></a>

## indexing_using_zigzag

In [166]:
zigzag_indexing = np.argsort(ind_zz)

def indexing_using_zigzag(block, zigzag_indexing):
    output = block.ravel()
    return output[zigzag_indexing]

In [164]:
arr = np.arange(64).reshape((8,8))
print("arr\n", arr)
print("indexing", rearrange_using_zigzag(arr, zigzag_indexing))

arr
 [[ 0  1  2  3  4  5  6  7]
 [ 8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23]
 [24 25 26 27 28 29 30 31]
 [32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47]
 [48 49 50 51 52 53 54 55]
 [56 57 58 59 60 61 62 63]]
indexing [ 0  8  1  2  9 16 24 17 10  3  4 11 18 25 32 40 33 26 19 12  5  6 13 20
 27 34 41 48 56 49 42 35 28 21 14  7 15 22 29 36 43 50 57 58 51 44 37 30
 23 31 38 45 52 59 60 53 46 39 47 54 61 62 55 63]


<a id="snr"></a>

## SNR

A relação sinal-ruído compara o nível de um sinal desejado com o nível do ruído. Quanto mais alta for a relação sinal-ruído, menor é o efeito do ruído de fundo sobre a detecção ou medição do sinal.

$$ SNR(Db) = 10\log_{10}\left[\frac{\sum_{l}\sum_{c}I_{ap}(l,c)^2}{\sum_{l}\sum_{c}[I_{ap}(l,c)-I_{or}(l,c)]^2}\right] $$

In [44]:
def snr(I_or, I_ap):
    Pxa = np.sum(I_ap.astype('float')**2)
    Pe = np.sum( (I_ap.astype('float') - I_or.astype('float'))**2 )
    return 10*np.log10(Pxa / Pe)

<a id="quality_factor"></a>

## quality_factor

Função para calcular o valor a multiplicar  a tabela K1 a partir do fator de qualidade.

In [45]:
def quality_factor(q):
    if(q <= 50):
        factor = 50.0 / q
    else:
        factor = 2.0 - (q * 2.0)/100.0
    return factor 

<a id="io_utilities"></a>

# I/O Utilities

<a id="count_bits"></a>

## count_bits

Função que conta o número de bits de um número.

In [124]:
def count_bits(number):
     return len(bin(number)[2:])

<a id="input_bit_reader"></a>

## InputBitReader

Para realizar compressão e descompressão com eficácia, é necessesário manipular os fluxos de dados como um fluxo de bits individuais. 

In [46]:
class InputBitReader(object): 
    def __init__(self, bit_seq): 
        self.bit_seq = bit_seq
        self.size = len(bit_seq)
        self.bits_read = 0
        self.buffer = []

    def read_bit(self):
        if self.bits_read < self.size:
            return self.read_bits(1)[0]
        else:
            return None

    def read_bits(self, n):
        self.__flush()
        if self.bits_read < self.size:
            self.buffer = pad_bits(self.bit_seq[self.bits_read:(self.bits_read + n)], n)
            self.bits_read += n
        return self.buffer
    
    def read_byte(self):
        if self.bits_read < self.size:
            byte = ''.join(list(map(str, self.read_bits(8))))
            return int(byte, 2)
        else:
            return None

    def __flush(self):
        self.buffer = []

<a id="divide_to_blocks"></a>

## divide_to_blocks

In [47]:
def divide_to_blocks(image, size):
    output = []
    
    if len(image.shape) < 3: 
        width, height = image.shape
        for lin in range(0, width, size):
            for col in range(0, height, size):
                output.append(image[lin:(lin+size),col:(col+size)])
    else:
        width, height, band = image.shape
        for bnd in range(0, band):
            for lin in range(0, width, size):
                for col in range(0, height, size):
                    output.append(image[lin:(lin+size),col:(col+size),bnd])
                
    return output, image.shape

In [48]:
lena_gray_blocks, lena_gray_shape  = divide_to_blocks(lena_gray, 8)
lena_color_blocks, lena_color_shape = divide_to_blocks(lena_color, 8)

In [49]:
print("number of blocks for lena gray:", len(lena_gray_blocks))
print("number of blocks for lena color:", len(lena_color_blocks))

number of blocks for lena gray: 4096
number of blocks for lena color: 12288


<a id="merge_from_blocks"></a>

## merge_from_blocks

In [50]:
def merge_from_blocks(arr, shape, size):
    output = np.zeros(shape, dtype='uint8')
    index = 0
    if len(shape) < 3:
        for lin in range(0, shape[0], size):
            for col in range(0, shape[1], size):
                output[lin:(lin+size),col:(col+size)] = arr[index]
                index = index + 1
    else:
        for bnd in range(0, shape[2]):
            for lin in range(0, shape[0], size):
                for col in range(0, shape[1], size):
                    output[lin:(lin+size),col:(col+size),bnd] = arr[index]
                    index = index + 1
    return output

In [51]:
re_lena_gray = merge_from_blocks(lena_gray_blocks, lena_gray_shape, 8)
re_lena_color = merge_from_blocks(lena_color_blocks, lena_color_shape, 8)

In [52]:
cv2.imshow("re_lena_gray", re_lena_gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imshow("re_lena_gray", re_lena_color)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [53]:
#arr[:,:,r] = [];

<a id="dct"></a>

# DCT

<a id="dct_encode"></a>

## DCT Encoder

In [54]:
def dct(block):
    return cv2.dct(np.float32(block)) # float conversion

In [55]:
block_dct = []
for i in range(len(lena_gray_blocks)):
    block_dct.append(dct(lena_gray_blocks[i]))

In [57]:
re_from_dct = merge_from_blocks(block_dct, lena_gray_shape, 8)
cv2.imshow("DCT", re_from_dct)
cv2.waitKey(0)
cv2.destroyAllWindows()

<a id="dct_decoder"></a>

## DCT Decoder

In [58]:
def idct(block):
    return cv2.idct(block)

In [59]:
block_idct = []
for i in range(len(block_dct)):
    block_idct.append(idct(block_dct[i]))

In [60]:
re_from_idct = merge_from_blocks(block_idct, lena_gray_shape, 8)
cv2.imshow("IDCT", re_from_idct)
cv2.waitKey(0)
cv2.destroyAllWindows()

[back to index](#index)

<a id="quantificacao"></a>

# Quantificação

As tabelas de quantifiacação resultam de estudos psico-visuais, com o objectivo de maximizar a taxa de compressão sem perda
perceptiveis de qualidade.

<a id="quantific_encode"></a>

## quantific_encode

In [61]:
def quantific_encode(block, k1, q):
    quant_block = np.zeros((8, 8))
    factor = quality_factor(q)
    
    # divide by k1  and multiply by quality factor
    for i in range(0,8):
        quant_block[i] = np.round(block[i] / (k1[i] * factor))
        
    return quant_block

<a id="quantific_decode"></a>

## quantific_decode

In [62]:
def quantific_decode(block, k1, q):
    iquant_block = np.zeros((8, 8))
    factor = quality_factor(q)
    
    # multily by k1 and divide by quality factor
    for i in range(0,8):
        iquant_block[i] = np.round(block[i] * (k1[i] / factor))
        
    return iquant_block

[back to index](#index)

<a id="dpcm"></a>

# DPCM

O valor do componente DC em cada bloco 8x8 é grande e varia entre os blocos, mas geralmente está próximo ao do bloco anterior.
Differential Pulse Code Modulation (DPCM): codifica a diferença entre o bloco 8x8 atual e o anterior, resultando num número menor -> menos bits

<a id="dpcm_encode"></a>

## dpcm_encode

In [89]:
def dpcm_encode(b1, b2):
    return b2[i][0,0] - b1[i - 1][0,0]

<a id="dpcm_decode"></a>

## dpcm_decode

In [122]:
def dpcm_encode(b1, b2):
    return b2[i][0,0] + b1[i - 1][0,0]

[back to index](#index)

<a id="rlc"></a>

# RLC

Codificação run-length é uma forma simples de compressão sem perda de dados onde sequências longas de valores repetidos são armazenadas como um único valor e sua contagem no lugar de sua sequência original.

<a id="rlc_encode"></a>

## rlc_encode

In [177]:
def rlc_encode(block, zigzag_indexing):
    # indexing using zigzag
    zzblock = indexing_using_zigzag(block, zigzag_indexing)
    
    rlc = []
    
    zrl = 0
    for i in range(1, len(zzblock)):
        ac = zzblock[i]
        if ac != 0:
            if zrl == 0:
                rlc.append( (zrl, ac) )
            else:
                rlc.append( (zrl, ac) )
                zrl = 0
        else:
            zrl += 1
            
    return rlc

In [178]:
def rlc_decode(rlc, zigzag_indexing):
    
    block = np.zeros(63)
    for i in range(0, len(rlc)):
        rlc[i]
        if ac != 0:
            if zrl == 0:
                rlc.append( (zrl, ac) )
            else:
                rlc.append( (zrl, ac) )
                zrl = 0
        else:
            zrl += 1
    


[(0, 162), (0, 162), (0, 162), (0, 162), (0, 162), (0, 162), (0, 162), (0, 162), (0, 161), (0, 162), (0, 161), (0, 162), (0, 162), (0, 162), (0, 164), (0, 162), (0, 162), (0, 161), (0, 162), (0, 157), (0, 163), (0, 157), (0, 162), (0, 161), (0, 162), (0, 164), (0, 160), (0, 159), (0, 160), (0, 158), (0, 161), (0, 162), (0, 157), (0, 163), (0, 161), (0, 161), (0, 163), (0, 157), (0, 162), (0, 155), (0, 163), (0, 159), (0, 155), (0, 158), (0, 161), (0, 157), (0, 163), (0, 161), (0, 161), (0, 163), (0, 159), (0, 160), (0, 157), (0, 158), (0, 162), (0, 159), (0, 161), (0, 160), (0, 159), (0, 159), (0, 156), (0, 156), (0, 157)]


In [183]:
print(lena_block_1)
dct = dct(lena_block_1)
print(dct)
quant = quantific_encode(dct,q,80)
print(quant)

[[162 162 162 161 162 157 163 161]
 [162 162 162 161 162 157 163 161]
 [162 162 162 161 162 157 163 161]
 [162 162 162 161 162 157 163 161]
 [162 162 162 161 162 157 163 161]
 [164 164 158 155 161 159 159 160]
 [160 160 163 158 160 162 159 156]
 [159 159 155 157 158 159 156 157]]


TypeError: 'numpy.ndarray' object is not callable

In [173]:
print(K5[(0,0)])
d = (0,9)
print(d)
print(d[1])
print(K5[d])

1010
(0, 9)
9
1111111110000010
